In [2]:
import numpy as np
#from netCDF4 import Dataset
import pygrib
import pandas as pd
import os, time, datetime
import glob
#http://www.himpactwxlab.com/home/how-to-wiki/write-grib2-data-with-pygrib

## Open a grib/grib2 file

In [3]:
file="gdas1.fnl0p25.2018060318.f06.grib2" #example filename
gr = pygrib.open(file)


## Print an inventory of file

In [4]:
for g in gr:
    print(g)

1:U component of wind:m s**-1 (instant):regular_ll:unknown:level 0:fcst time 6 hrs:from 201806031800
2:V component of wind:m s**-1 (instant):regular_ll:unknown:level 0:fcst time 6 hrs:from 201806031800
3:Ventilation Rate:m**2 s**-1 (instant):regular_ll:unknown:level 0:fcst time 6 hrs:from 201806031800
4:Wind speed (gust):m s**-1 (instant):regular_ll:surface:level 0:fcst time 6 hrs:from 201806031800
5:Geopotential Height:gpm (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 6 hrs:from 201806031800
6:Temperature:K (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 6 hrs:from 201806031800
7:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 6 hrs:from 201806031800
8:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 6 hrs:from 201806031800
9:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 6 hrs:from 201806031800
10:Ozone mixing ratio:kg kg**-1 (instant):regular_ll:isobaricInh

311:Convective inhibition:J kg**-1 (instant):regular_ll:surface:level 0:fcst time 6 hrs:from 201806031800
312:Precipitable water:kg m**-2 (instant):regular_ll:unknown:level 0 considered as a single layer:fcst time 6 hrs:from 201806031800
313:Cloud water:kg m**-2 (instant):regular_ll:unknown:level 0 considered as a single layer:fcst time 6 hrs:from 201806031800
314:Relative humidity:% (instant):regular_ll:unknown:level 0 considered as a single layer:fcst time 6 hrs:from 201806031800
315:Total ozone:DU (instant):regular_ll:unknown:level 0 considered as a single layer:fcst time 6 hrs:from 201806031800
316:Total Cloud Cover:% (avg):regular_ll:unknown:level 0:fcst time 0-6 hrs (avg):from 201806031800
317:Total Cloud Cover:% (avg):regular_ll:unknown:level 0:fcst time 0-6 hrs (avg):from 201806031800
318:Total Cloud Cover:% (avg):regular_ll:unknown:level 0:fcst time 0-6 hrs (avg):from 201806031800
319:Total Cloud Cover:% (avg):regular_ll:atmosphere:level 0 -:fcst time 0-6 hrs (avg):from 201806

The process will print out each record stored in the file, similar to the output you would see from using the wgrib/wgrib2 programs.

To print some more detailed variable attributes, such as variable name, level, coordinate level, valid date, etc. :

In [5]:
for g in gr:
  print(g.typeOfLevel, g.level, g.name, g.validDate, g.analDate, g.forecastTime)

When using pygrib (which uses ECMWF grip API), there a number of different keywords that you can use. Many of these will be specific to the content within the grib message (e.g. see https://software.ecmwf.int/wiki/display/GRIB/Grib+API+keys). If you want to know which keys are available, , you can use grib_dump filename.grb to dump the contents of the grib file (see: https://software.ecmwf.int/wiki/display/GRIB/grib_dump) and see all of the possible keywords.

### Get a list of N grib messages

To obtain a list of the first N number of grib messages, we can use read method. For example, to get a list of the first 10 grib messages:


In [6]:
grbmsgs = gr.read(10)

OSError: not that many messages in file

To get a list of grib messages that have a certain name, level, etc. we can use the select method with optional keyword arguments. For example, if we want to obtain a list of all the grib messages that consist of temperature on pressure levels, we could do this:


In [7]:
tmpmsgs = gr.select(name='Temperature',typeOfLevel='isobaricInhPa')

In [8]:
tmpmsgs

[6:Temperature:K (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 6 hrs:from 201806031800,
 12:Temperature:K (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 6 hrs:from 201806031800,
 18:Temperature:K (instant):regular_ll:isobaricInhPa:level 300 Pa:fcst time 6 hrs:from 201806031800,
 24:Temperature:K (instant):regular_ll:isobaricInhPa:level 500 Pa:fcst time 6 hrs:from 201806031800,
 30:Temperature:K (instant):regular_ll:isobaricInhPa:level 700 Pa:fcst time 6 hrs:from 201806031800,
 36:Temperature:K (instant):regular_ll:isobaricInhPa:level 1000 Pa:fcst time 6 hrs:from 201806031800,
 43:Temperature:K (instant):regular_ll:isobaricInhPa:level 2000 Pa:fcst time 6 hrs:from 201806031800,
 50:Temperature:K (instant):regular_ll:isobaricInhPa:level 3000 Pa:fcst time 6 hrs:from 201806031800,
 57:Temperature:K (instant):regular_ll:isobaricInhPa:level 5000 Pa:fcst time 6 hrs:from 201806031800,
 64:Temperature:K (instant):regular_ll:isobaricInhPa:level 7000 Pa:fcst time 6 hrs:from 

In [9]:
#get message 38 (temp @70000pa in this example)
msg = gr[186] #get record number 38
print(msg)

186:Temperature:K (instant):regular_ll:isobaricInhPa:level 70000 Pa:fcst time 6 hrs:from 201806031800


### Get values associated with the grib message
    Now, we want to get the values that are associated with the grib message we have just read in

In [10]:
temp_vals = msg.values

 Values are read into a NumPy array named temp_vals. We may now access in Values are read into a NumPy array named temp_vals. We may now access information about the array, such as the shape and max/min values, as is normally done with numpy arrays.formation about the array, such as the shape and max/min values, as is normally done with numpy arrays.

In [11]:
print(temp_vals.shape)
print(np.amax(temp_vals),np.amin(temp_vals))

(721, 1440)
293.66486 225.06485


To get latitude/longitude values associated with the grib message, we use the latlons() method:

In [12]:
lats,lons = msg.latlons()

## Grib2 Decode
    If you have read in a grib2 file, and you want to obtain information about the encoding specifics, you must import the ncepgrib2 module, specifically the Grib2Decode class. Ncepgrib2 is a wrapper around the NCEP grib2 C library.


In [13]:
#import Grib2Decode from ncepgrib2
from ncepgrib2 import Grib2Decode

#get info about grib2 section specifications
grb = Grib2Decode(msg.tostring(),gribmsg=True)
print(grb)

AttributeError: 'dict_keys' object has no attribute 'sort'

References: 
http://pygrib.googlecode.com/svn/trunk/docs/index.html
http://pygrib.googlecode.com/svn/trunk/ncepgrib2_docs/ncepgrib2-module.html#Grib2Decode
http://www.pyngl.ucar.edu/Nio.shtml
http://scitools.org.uk/iris/